In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PROJECT_DIR = "/content/drive/MyDrive/quarter_close_alm"

In [7]:
!pip -q install pandas numpy pyyaml openpyxl

import os

folders = ["config","outputs","src",]

for f in folders:
    os.makedirs(os.path.join(PROJECT_DIR, f), exist_ok=True)

In [6]:
config_text = """\
run_id: "2025Q4_MVP"
base_date: "2025-12-31"
scenario:
  name: "RatesUp100_SpreadsUp50"
  rate_shock_bps: 100
  spread_shock_bps: 50
paths:
  outputs_root: "outputs"
"""

config_path = os.path.join(PROJECT_DIR, "config", "run_config.yml")
with open(config_path, "w") as f:
    f.write(config_text)

In [10]:
import pandas as pd
import numpy as np

def generate_synthetic_holdings(n=50, seed=7):
    rng = np.random.default_rng(seed)
    ratings = ["AAA","AA","A","BBB","BB"]
    sectors = ["Corp","Govt","MBS","ABS"]
    df = pd.DataFrame({
        "asset_id": [f"A{str(i).zfill(4)}" for i in range(1, n+1)],
        "cusip": [f"{rng.integers(100000000,999999999)}" for _ in range(n)],
        "asset_type": ["bond"]*n,
        "market_value": rng.uniform(0.5e6, 5e6, size=n).round(2),
        "book_value": rng.uniform(0.5e6, 5e6, size=n).round(2),
        "mod_duration": rng.uniform(1.0, 8.0, size=n).round(3),
        "oas_spread_bps": rng.integers(50, 350, size=n),
        "coupon_rate": rng.uniform(0.02, 0.07, size=n).round(4),
        "maturity_years": rng.integers(1, 30, size=n),
        "rating": rng.choice(ratings, size=n),
        "sector": rng.choice(sectors, size=n),
    })
    # MV around BV
    df["market_value"] = (df["book_value"] * rng.uniform(0.92, 1.08, size=n)).round(2)
    return df

def make_holdings_messy(df, seed=11, messiness=0.25):
    """
    messiness ~ fraction of rows impacted by issues (0.0 to 0.6 recommended)
    Produces realistic dirty data patterns seen in quarter-close feeds.
    """
    rng = np.random.default_rng(seed)
    df = df.copy()

    n = len(df)
    k = max(1, int(n * messiness))
    idx = rng.choice(df.index, size=k, replace=False)

    # 1) Missing / blank asset_id
    miss_id = rng.choice(idx, size=max(1, k//6), replace=False)
    df.loc[miss_id, "asset_id"] = rng.choice([None, "", "   "], size=len(miss_id))

    # 2) Duplicate asset_id (copy a few IDs to other rows)
    dup_targets = rng.choice(df.index, size=max(1, k//8), replace=False)
    dup_sources = rng.choice(df.index, size=len(dup_targets), replace=False)
    df.loc[dup_targets, "asset_id"] = df.loc[dup_sources, "asset_id"].values

    # 3) Non-numeric market_value (commas, $, text)
    bad_mv = rng.choice(idx, size=max(1, k//6), replace=False)
    df.loc[bad_mv, "market_value"] = df.loc[bad_mv, "market_value"].apply(
        lambda x: rng.choice([f"${x:,.2f}", f"{x:,.0f}", "N/A", "missing"])
    )

    # 4) Negative or zero values
    neg_mv = rng.choice(idx, size=max(1, k//10), replace=False)
    df.loc[neg_mv, "market_value"] = rng.choice([0, -1000, -500000], size=len(neg_mv))

    # 5) Duration missing / string duration
    bad_dur = rng.choice(idx, size=max(1, k//6), replace=False)
    df.loc[bad_dur, "mod_duration"] = rng.choice([None, "", "5.2", "ten"], size=len(bad_dur))

    # 6) Rating inconsistencies (case, spaces, unmapped values)
    bad_rat = rng.choice(idx, size=max(1, k//5), replace=False)
    df.loc[bad_rat, "rating"] = rng.choice(
        ["bbb", "BBB ", "AA-", "NR", "Not Rated", None, "A ", "BB+"],
        size=len(bad_rat)
    )

    # 7) Sector missing / messy labels
    bad_sec = rng.choice(idx, size=max(1, k//7), replace=False)
    df.loc[bad_sec, "sector"] = rng.choice([None, "", "Corp ", "GOVT", "AssetBacked"], size=len(bad_sec))

    # 8) CUSIP formatting issues
    bad_cusip = rng.choice(idx, size=max(1, k//7), replace=False)
    df.loc[bad_cusip, "cusip"] = df.loc[bad_cusip, "cusip"].astype(str).apply(
        lambda s: rng.choice([s, f"{s}-", f" {s} ", s[:6], "UNKNOWN"])
    )

    # 9) Asset type inconsistencies
    bad_type = rng.choice(idx, size=max(1, k//8), replace=False)
    df.loc[bad_type, "asset_type"] = rng.choice(["Bond", "BOND", "bond ", "cash", None], size=len(bad_type))

    return df

# Example usage:
clean_holdings = generate_synthetic_holdings(n=60, seed=7)
messy_holdings = make_holdings_messy(clean_holdings, seed=11, messiness=0.30)

messy_holdings.head(10)


/tmp/ipython-input-2022624037.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[np.str_('N/A') np.str_('missing') np.str_('missing')]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[bad_mv, "market_value"] = df.loc[bad_mv, "market_value"].apply(
/tmp/ipython-input-2022624037.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[None None 'ten']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[bad_dur, "mod_duration"] = rng.choice([None, "", "5.2", "ten"], size=len(bad_dur))


,asset_id,cusip,asset_type,market_value,book_value,mod_duration,oas_spread_bps,coupon_rate,maturity_years,rating,sector
0,A0001,950414459,bond,638561.73,613385.92,3.329,110,0.0267,12,AA,MBS
1,A0002,715761942,bond,missing,2174833.73,3.788,248,0.0531,5,BB,
2,A0003,907492419,bond,679298.83,636576.32,2.42,308,0.0615,11,BBB,Govt
3,A0004,620463206,bond,-500000,1053014.46,1.355,180,0.0388,10,AAA,ABS
4,A0005,798117120,bond,4719748.79,4852167.06,2.49,289,0.0386,13,AAA,ABS
5,A0006,850285918,bond,3701643.58,3459923.29,7.408,141,0.0470,14,AA,Corp
6,A0007,302686470,bond,2293071.79,2426991.11,6.881,88,0.0308,1,A,ABS
7,A0008,149978419,bond,2863442.95,2856830.49,1.787,298,0.0324,10,AAA,Govt
8,A0009,356560330,bond,4138289.1,4427641.44,5.226,280,0.0365,25,AAA,Corp
9,A0010,886198099,bond,2201529.95,2048948.00,4.354,275,0.0429,13,AAA,Govt


In [11]:
import os

raw_path = os.path.join(PROJECT_DIR, "data", "synthetic")
os.makedirs(raw_path, exist_ok=True)

messy_file = os.path.join(raw_path, "holdings_messy.csv")
messy_holdings.to_csv(messy_file, index=False)
messy_file

'/content/drive/MyDrive/quarter_close_alm/data/synthetic/holdings_messy.csv'

In [17]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/quarter_close_alm"

import os
os.makedirs(os.path.join(PROJECT_DIR, "config"), exist_ok=True)
os.makedirs(os.path.join(PROJECT_DIR, "data", "synthetic"), exist_ok=True)
os.makedirs(os.path.join(PROJECT_DIR, "outputs"), exist_ok=True)

print("PROJECT_DIR:", PROJECT_DIR)
print("Exists:", os.path.exists(PROJECT_DIR))
print("Config folder:", os.path.join(PROJECT_DIR, "config"))
print("Data folder:", os.path.join(PROJECT_DIR, "data", "synthetic"))
print("Outputs folder:", os.path.join(PROJECT_DIR, "outputs"))




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROJECT_DIR: /content/drive/MyDrive/quarter_close_alm
Exists: True
Config folder: /content/drive/MyDrive/quarter_close_alm/config
Data folder: /content/drive/MyDrive/quarter_close_alm/data/synthetic
Outputs folder: /content/drive/MyDrive/quarter_close_alm/outputs


In [18]:
import yaml

config_path = os.path.join(PROJECT_DIR, "config", "run_config.yml")
assert os.path.exists(config_path), f"Missing config: {config_path}"

with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

run_id = cfg["run_id"]
out_root = cfg["paths"]["outputs_root"]

run_path = os.path.join(PROJECT_DIR, out_root, run_id)
axis_path = os.path.join(run_path, "axis_like_inputs")

os.makedirs(axis_path, exist_ok=True)

data_syn = os.path.join(PROJECT_DIR, "data", "synthetic")
os.makedirs(data_syn, exist_ok=True)

print("RUN_ID:", run_id)
print("RUN_PATH:", run_path)
print("AXIS_PATH:", axis_path)
print("DATA_SYN:", data_syn)


RUN_ID: 2025Q4_MVP
RUN_PATH: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP
AXIS_PATH: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/axis_like_inputs
DATA_SYN: /content/drive/MyDrive/quarter_close_alm/data/synthetic


In [19]:
import pandas as pd
import numpy as np

def generate_base_assumptions():
    yield_curve = pd.DataFrame({
        "tenor_years": [1,2,3,5,7,10,20,30],
        "zero_rate":   [0.048,0.046,0.045,0.044,0.0435,0.043,0.042,0.0415],
    })
    credit_spreads = pd.DataFrame({
        "rating": ["AAA","AA","A","BBB","BB","B"],
        "spread_bps": [25, 45, 75, 140, 260, 420],
    })
    return yield_curve, credit_spreads

def apply_shocks(yield_curve, credit_spreads, rate_shock_bps, spread_shock_bps):
    yc = yield_curve.copy()
    cs = credit_spreads.copy()
    yc["zero_rate"] = yc["zero_rate"] + rate_shock_bps/10000.0
    cs["spread_bps"] = cs["spread_bps"] + spread_shock_bps
    return yc, cs

yield_curve_base, credit_spreads_base = generate_base_assumptions()

rate_shock = cfg["scenario"]["rate_shock_bps"]
spread_shock = cfg["scenario"]["spread_shock_bps"]

yield_curve_shocked, credit_spreads_shocked = apply_shocks(
    yield_curve_base, credit_spreads_base, rate_shock, spread_shock
)

# Save "source-style" files
yield_curve_base.to_csv(os.path.join(data_syn, "yield_curve_base.csv"), index=False)
credit_spreads_base.to_csv(os.path.join(data_syn, "credit_spreads_base.csv"), index=False)
yield_curve_shocked.to_csv(os.path.join(data_syn, "yield_curve_shocked.csv"), index=False)
credit_spreads_shocked.to_csv(os.path.join(data_syn, "credit_spreads_shocked.csv"), index=False)

yield_curve_base, credit_spreads_base


(   tenor_years  zero_rate
 0            1     0.0480
 1            2     0.0460
 2            3     0.0450
 3            5     0.0440
 4            7     0.0435
 5           10     0.0430
 6           20     0.0420
 7           30     0.0415,
   rating  spread_bps
 0    AAA          25
 1     AA          45
 2      A          75
 3    BBB         140
 4     BB         260
 5      B         420)

In [20]:
# Assumption files in AXIS-like folder
yield_curve_base.to_csv(os.path.join(axis_path, "econ_yield_curve_base.csv"), index=False)
yield_curve_shocked.to_csv(os.path.join(axis_path, "econ_yield_curve_shocked.csv"), index=False)
credit_spreads_base.to_csv(os.path.join(axis_path, "credit_spreads_base.csv"), index=False)
credit_spreads_shocked.to_csv(os.path.join(axis_path, "credit_spreads_shocked.csv"), index=False)

# Minimal investment input rules (MVP)
investment_inputs = pd.DataFrame([
    {"rule": "liquidity_reserve", "min_cash_pct": 0.03},
    {"rule": "reinvest_cash", "target_sector": "Corp", "target_rating": "A", "pct": 0.60},
    {"rule": "reinvest_cash", "target_sector": "Govt", "target_rating": "AA", "pct": 0.40},
])
investment_inputs.to_csv(os.path.join(axis_path, "investment_inputs.csv"), index=False)

print("Wrote AXIS-like assumptions + investment inputs to:", axis_path)
sorted(os.listdir(axis_path))


Wrote AXIS-like assumptions + investment inputs to: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/axis_like_inputs


['asset_inventory.csv',
 'credit_spreads.csv',
 'credit_spreads_base.csv',
 'credit_spreads_shocked.csv',
 'econ_yield_curve.csv',
 'econ_yield_curve_base.csv',
 'econ_yield_curve_shocked.csv',
 'investment_inputs.csv']

In [21]:
import re

messy_file = os.path.join(data_syn, "holdings_messy.csv")
assert os.path.exists(messy_file), f"Missing: {messy_file}. Save your messy holdings there."

raw = pd.read_csv(messy_file, dtype=str)
raw.to_csv(os.path.join(axis_path, "asset_inventory_raw.csv"), index=False)

def clean_currency_to_float(x):
    if x is None:
        return np.nan
    s = str(x).strip()
    if s == "" or s.lower() in ["na","n/a","none","null","missing"]:
        return np.nan
    s = re.sub(r"[\$,]", "", s)
    try:
        return float(s)
    except:
        return np.nan

def clean_float(x):
    if x is None:
        return np.nan
    s = str(x).strip()
    if s == "" or s.lower() in ["na","n/a","none","null","missing"]:
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

def normalize_text(x):
    if x is None:
        return ""
    return str(x).strip()

def normalize_rating(x):
    s = normalize_text(x).upper().replace(" ", "")
    mapping = {
        "AA-":"AA","AA+":"AA",
        "A-":"A","A+":"A",
        "BBB-":"BBB","BBB+":"BBB",
        "BB-":"BB","BB+":"BB",
        "NOTRATED":"NR","NOTRATED.":"NR","NR":"NR"
    }
    return mapping.get(s, s)

def normalize_sector(x):
    s = normalize_text(x).upper().replace(" ", "")
    mapping = {"GOVT":"GOVT","GOVERNMENT":"GOVT","CORP":"CORP","MBS":"MBS","ABS":"ABS","ASSETBACKED":"ABS"}
    return mapping.get(s, s)

def normalize_asset_type(x):
    s = normalize_text(x).lower().strip()
    if s in ["bond","bonds"]:
        return "bond"
    if s in ["cash"]:
        return "cash"
    return s

clean = pd.DataFrame()
clean["asset_id"] = raw.get("asset_id", "").apply(normalize_text)
clean["cusip"] = raw.get("cusip", "").apply(normalize_text)
clean["asset_type"] = raw.get("asset_type", "").apply(normalize_asset_type)
clean["market_value"] = raw.get("market_value", "").apply(clean_currency_to_float)
clean["book_value"] = raw.get("book_value", "").apply(clean_currency_to_float)
clean["mod_duration"] = raw.get("mod_duration", "").apply(clean_float)
clean["oas_spread_bps"] = raw.get("oas_spread_bps", "").apply(clean_float)
clean["coupon_rate"] = raw.get("coupon_rate", "").apply(clean_float)
clean["maturity_years"] = raw.get("maturity_years", "").apply(clean_float)
clean["rating"] = raw.get("rating", "").apply(normalize_rating)
clean["sector"] = raw.get("sector", "").apply(normalize_sector)

clean.to_csv(os.path.join(axis_path, "asset_inventory.csv"), index=False)

print("Saved asset inventory files to AXIS-like folder.")
clean.head(8)


Saved asset inventory files to AXIS-like folder.


,asset_id,cusip,asset_type,market_value,book_value,mod_duration,oas_spread_bps,coupon_rate,maturity_years,rating,sector
0,A0001,950414459,bond,638561.73,613385.92,3.329,110.0,0.0267,12.0,AA,MBS
1,A0002,715761942,bond,NaN,2174833.73,3.788,248.0,0.0531,5.0,BB,NAN
2,A0003,907492419,bond,679298.83,636576.32,2.420,308.0,0.0615,11.0,BBB,GOVT
3,A0004,620463206,bond,-500000.00,1053014.46,1.355,180.0,0.0388,10.0,AAA,ABS
4,A0005,798117120,bond,4719748.79,4852167.06,2.490,289.0,0.0386,13.0,AAA,ABS
5,A0006,850285918,bond,3701643.58,3459923.29,7.408,141.0,0.0470,14.0,AA,CORP
6,A0007,302686470,bond,2293071.79,2426991.11,6.881,88.0,0.0308,1.0,A,ABS
7,A0008,149978419,bond,2863442.95,2856830.49,1.787,298.0,0.0324,10.0,AAA,GOVT


In [23]:
import os
import pandas as pd
import numpy as np

# Paths (should already exist from earlier steps)
recon_path = os.path.join(run_path, "reconciliation_pack")
os.makedirs(recon_path, exist_ok=True)

asset_file = os.path.join(axis_path, "asset_inventory.csv")
yc_base_file = os.path.join(axis_path, "econ_yield_curve_base.csv")
yc_shock_file = os.path.join(axis_path, "econ_yield_curve_shocked.csv")
cs_base_file = os.path.join(axis_path, "credit_spreads_base.csv")
cs_shock_file = os.path.join(axis_path, "credit_spreads_shocked.csv")

for p in [asset_file, yc_base_file, yc_shock_file, cs_base_file, cs_shock_file]:
    assert os.path.exists(p), f"Missing required file: {p}"

assets = pd.read_csv(asset_file)
yc_base = pd.read_csv(yc_base_file)
yc_shock = pd.read_csv(yc_shock_file)
cs_base = pd.read_csv(cs_base_file)
cs_shock = pd.read_csv(cs_shock_file)

# ---------- Phase 1.5: Exceptions (realistic MVP checks) ----------
exceptions_rows = []

def add_exc(code, detail, sample_ids=None):
    exceptions_rows.append({
        "issue_code": code,
        "issue_detail": detail,
        "sample_asset_ids": ", ".join(sample_ids) if sample_ids else ""
    })

# 1) Missing / blank asset_id
missing_id = assets[assets["asset_id"].isna() | (assets["asset_id"].astype(str).str.strip() == "")]
if len(missing_id) > 0:
    add_exc("MISSING_ASSET_ID", f"{len(missing_id)} rows missing/blank asset_id",
            list(missing_id.head(5)["asset_id"].astype(str)))

# 2) Duplicate asset_id (ignore blanks)
nonblank = assets[assets["asset_id"].astype(str).str.strip() != ""].copy()
dup = nonblank[nonblank["asset_id"].duplicated(keep=False)]
if len(dup) > 0:
    add_exc("DUPLICATE_ASSET_ID", f"{dup['asset_id'].nunique()} duplicated asset_id values",
            list(dup["asset_id"].dropna().astype(str).unique()[:5]))

# 3) Market value invalid
bad_mv = assets[assets["market_value"].isna() | (assets["market_value"] <= 0)]
if len(bad_mv) > 0:
    add_exc("INVALID_MARKET_VALUE", f"{len(bad_mv)} rows with MV missing/<=0",
            list(bad_mv.head(5)["asset_id"].astype(str)))

# 4) Duration missing for bonds
bond = assets[assets["asset_type"].astype(str).str.lower().str.strip() == "bond"]
bad_dur = bond[bond["mod_duration"].isna()]
if len(bad_dur) > 0:
    add_exc("MISSING_DURATION", f"{len(bad_dur)} bond rows missing duration",
            list(bad_dur.head(5)["asset_id"].astype(str)))

# 5) Rating not mapped in spreads base
valid_ratings = set(cs_base["rating"].astype(str).str.upper().str.strip())
assets_r = assets.copy()
assets_r["rating_norm"] = assets_r["rating"].astype(str).str.upper().str.strip()
bad_rating = assets_r[~assets_r["rating_norm"].isin(valid_ratings)]
# Only flag if rating is present
bad_rating = bad_rating[bad_rating["rating_norm"] != ""]
if len(bad_rating) > 0:
    add_exc("RATING_NOT_MAPPED", f"{len(bad_rating)} rows with rating not in spreads table",
            list(bad_rating.head(5)["asset_id"].astype(str)))

exceptions = pd.DataFrame(exceptions_rows)
exceptions_file = os.path.join(recon_path, "exceptions_assets.csv")
exceptions.to_csv(exceptions_file, index=False)

# ---------- Phase 1.4: Base + Scenario valuation ----------
# Assets base MV
base_assets_mv = float(assets["market_value"].fillna(0).sum())

# Scenario asset MV via duration shock: ΔPrice% ≈ -Dur * Δy
rate_shock_bps = cfg["scenario"]["rate_shock_bps"]
spread_shock_bps = cfg["scenario"]["spread_shock_bps"]
dy = (rate_shock_bps + spread_shock_bps) / 10000.0

dur = assets["mod_duration"].fillna(0)
scen_assets_mv = float((assets["market_value"].fillna(0) * (1.0 - dur * dy)).sum())

# Liabilities: synthetic cashflows discounted by curve
def liability_pv(yield_curve_df):
    # 10 annual payments of 10m (simple proxy)
    t = np.arange(1, 11)
    cf = np.array([10_000_000]*10, dtype=float)

    ten = yield_curve_df["tenor_years"].values
    r = yield_curve_df["zero_rate"].values
    # interpolate
    r_t = np.interp(t, ten, r)
    df = 1 / ((1 + r_t) ** t)
    return float((cf * df).sum())

base_liab_pv = liability_pv(yc_base)
scen_liab_pv = liability_pv(yc_shock)

base_surplus = base_assets_mv - base_liab_pv
scen_surplus = scen_assets_mv - scen_liab_pv

kpis = pd.DataFrame([
    {"metric": "Assets_MV", "base": base_assets_mv, "scenario": scen_assets_mv},
    {"metric": "Liability_PV", "base": base_liab_pv, "scenario": scen_liab_pv},
    {"metric": "Surplus", "base": base_surplus, "scenario": scen_surplus},
    {"metric": "Scenario_Name", "base": cfg["scenario"]["name"], "scenario": cfg["scenario"]["name"]},
])

kpis_file = os.path.join(recon_path, "kpis.csv")
kpis.to_csv(kpis_file, index=False)

# ---------- Phase 1.5: Rollforward tables (MVP) ----------
def rollforward(begin, end, label):
    return pd.DataFrame([
        {"item": f"{label}_begin", "amount": begin},
        {"item": f"{label}_market_move", "amount": end - begin},
        {"item": f"{label}_end", "amount": end},
    ])

# Create a proxy "begin" balance so rollforward exists even in MVP
begin_assets = base_assets_mv * 0.985
begin_liab = base_liab_pv * 1.01

rf_assets = rollforward(begin_assets, base_assets_mv, "assets")
rf_liab = rollforward(begin_liab, base_liab_pv, "liability")

rf_assets_file = os.path.join(recon_path, "rollforward_assets.csv")
rf_liab_file = os.path.join(recon_path, "rollforward_liability.csv")

rf_assets.to_csv(rf_assets_file, index=False)
rf_liab.to_csv(rf_liab_file, index=False)

print("✅ Phase 1.4 complete: valuation done")
print("✅ Phase 1.5 complete: reconciliation_pack written")
print("Reconciliation pack contents:", sorted(os.listdir(recon_path)))
kpis


✅ Phase 1.4 complete: valuation done
✅ Phase 1.5 complete: reconciliation_pack written
Reconciliation pack contents: ['exceptions_assets.csv', 'kpis.csv', 'rollforward_assets.csv', 'rollforward_liability.csv']


,metric,base,scenario
0,Assets_MV,173414144.77,162966361.106892
1,Liability_PV,79588566.058964,75797168.329728
2,Surplus,93825578.711036,87169192.777164
3,Scenario_Name,RatesUp100_SpreadsUp50,RatesUp100_SpreadsUp50


In [24]:
import os
import pandas as pd

dash_path = os.path.join(run_path, "kpi_dashboard.xlsx")

# Load the outputs we just wrote
kpis = pd.read_csv(os.path.join(recon_path, "kpis.csv"))
rf_assets = pd.read_csv(os.path.join(recon_path, "rollforward_assets.csv"))
rf_liab = pd.read_csv(os.path.join(recon_path, "rollforward_liability.csv"))
exceptions = pd.read_csv(os.path.join(recon_path, "exceptions_assets.csv"))

with pd.ExcelWriter(dash_path, engine="openpyxl") as xl:
    kpis.to_excel(xl, sheet_name="Summary_KPIs", index=False)
    rf_assets.to_excel(xl, sheet_name="Rollforward_Assets", index=False)
    rf_liab.to_excel(xl, sheet_name="Rollforward_Liability", index=False)
    exceptions.to_excel(xl, sheet_name="Exceptions", index=False)

print(dash_path)

from google.colab import files
files.download(dash_path)


✅ Phase 1.6 complete: KPI dashboard created at:
/content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/kpi_dashboard.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
# Phase 2 — Controls + Data Validation (Simplified)
# Output: outputs/<run_id>/reconciliation_pack/exceptions_assets.csv
# Assumes you already have:
# - config/run_config.yml
# - outputs/<run_id>/axis_like_inputs/asset_inventory.csv
# - outputs/<run_id>/axis_like_inputs/credit_spreads_base.csv

!pip -q install pyyaml pandas numpy

import os, yaml
import pandas as pd
import numpy as np
from datetime import datetime

# ====== 0) Set PROJECT_DIR (Google Drive) ======
PROJECT_DIR = "/content/drive/MyDrive/quarter_close_alm"

# ====== 1) Load config + paths ======
config_path = os.path.join(PROJECT_DIR, "config", "run_config.yml")
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

run_id = cfg["run_id"]
out_root = cfg.get("paths", {}).get("outputs_root", "outputs")

run_path  = os.path.join(PROJECT_DIR, out_root, run_id)
axis_path = os.path.join(run_path, "axis_like_inputs")
recon_path = os.path.join(run_path, "reconciliation_pack")
os.makedirs(recon_path, exist_ok=True)

asset_file = os.path.join(axis_path, "asset_inventory.csv")
spreads_file = os.path.join(axis_path, "credit_spreads_base.csv")

assert os.path.exists(asset_file), f"Missing: {asset_file}"
assert os.path.exists(spreads_file), f"Missing: {spreads_file}"

assets = pd.read_csv(asset_file)
spreads = pd.read_csv(spreads_file)

N = len(assets)

# ====== 2) Define Phase 2 rules (simple) ======
REQUIRED_COLS = [
    "asset_id","cusip","asset_type","market_value","book_value",
    "mod_duration","rating","sector"
]

ALLOWED_ASSET_TYPES = {"bond", "cash"}
DURATION_MAX = 50
COUPON_MAX = 0.25
MATURITY_MAX = 100
MV_BV_OUTLIER_WARN = 0.50  # 50%

valid_ratings = set(spreads["rating"].astype(str).str.upper().str.strip())

# ====== 3) Exception writer helper ======
exceptions = []

def add_exc(severity, issue_code, detail, idx=None, sample_col="asset_id", fix=""):
    affected = int(len(idx)) if idx is not None else 0
    pct = 0.0 if N == 0 else round(affected / N, 6)
    sample_ids = ""
    if idx is not None and len(idx) > 0 and sample_col in assets.columns:
        sample_ids = ", ".join(
            assets.loc[idx, sample_col].astype(str).head(5).tolist()
        )[:200]
    exceptions.append({
        "run_id": run_id,
        "severity": severity,          # ERROR or WARN
        "issue_code": issue_code,
        "issue_detail": detail,
        "affected_rows": affected,
        "pct_of_file": pct,
        "sample_asset_ids": sample_ids,
        "suggested_fix": fix
    })

# ====== 4) Run checks ======

# 4.1 Schema checks
missing_cols = [c for c in REQUIRED_COLS if c not in assets.columns]
if missing_cols:
    add_exc(
        "ERROR",
        "MISSING_COLUMN",
        f"Missing required columns: {missing_cols}",
        idx=None,
        fix="Update upstream mapping so required fields exist before running."
    )

# If schema is broken, still write exceptions and stop (nothing else is reliable)
if missing_cols:
    out = pd.DataFrame(exceptions)
    out_path = os.path.join(recon_path, "exceptions_assets.csv")
    out.to_csv(out_path, index=False)
    raise RuntimeError(f"Schema failed. Wrote exceptions to: {out_path}")

# 4.2 Key checks
blank_id = assets[assets["asset_id"].isna() | (assets["asset_id"].astype(str).str.strip()=="")].index
if len(blank_id) > 0:
    add_exc("ERROR","MISSING_ASSET_ID",f"{len(blank_id)} rows have blank asset_id.", blank_id,
            fix="Populate asset_id for every row (unique key).")

tmp = assets.copy()
tmp["asset_id_norm"] = tmp["asset_id"].astype(str).str.strip()
dups = tmp[(tmp["asset_id_norm"]!="") & (tmp["asset_id_norm"].duplicated(keep=False))].index
if len(dups) > 0:
    add_exc("ERROR","DUPLICATE_ASSET_ID",
            f"{tmp.loc[dups,'asset_id_norm'].nunique()} duplicated asset_id values found.",
            dups,
            fix="De-duplicate upstream or add lot/position suffix to make keys unique.")

# 4.3 Numeric sanity (core)
bad_mv = assets[assets["market_value"].isna() | (assets["market_value"] <= 0)].index
if len(bad_mv) > 0:
    add_exc("ERROR","INVALID_MARKET_VALUE",
            f"{len(bad_mv)} rows have market_value missing or <= 0.", bad_mv,
            fix="Fix pricing feed / ensure market_value parses to a positive number.")

bad_bv = assets[assets["book_value"].isna() | (assets["book_value"] <= 0)].index
if len(bad_bv) > 0:
    add_exc("WARN","INVALID_BOOK_VALUE",
            f"{len(bad_bv)} rows have book_value missing or <= 0.", bad_bv,
            fix="Fix accounting feed / ensure book_value parses correctly.")

bad_dur = assets[assets["mod_duration"].isna() | (assets["mod_duration"] < 0) | (assets["mod_duration"] > DURATION_MAX)].index
if len(bad_dur) > 0:
    add_exc("ERROR","INVALID_DURATION",
            f"{len(bad_dur)} rows have duration missing/negative/> {DURATION_MAX}.", bad_dur,
            fix="Fix duration feed; ensure numeric years, not text/blank.")

# Optional numeric sanity (warn-level)
if "coupon_rate" in assets.columns:
    bad_coupon = assets[(assets["coupon_rate"].notna()) & ((assets["coupon_rate"] < 0) | (assets["coupon_rate"] > COUPON_MAX))].index
    if len(bad_coupon) > 0:
        add_exc("WARN","COUPON_OUT_OF_RANGE",
                f"{len(bad_coupon)} rows have coupon_rate outside 0–{COUPON_MAX}.", bad_coupon,
                fix="Confirm coupon is decimal (0.05), not percent (5).")

if "maturity_years" in assets.columns:
    bad_mat = assets[(assets["maturity_years"].notna()) & ((assets["maturity_years"] < 0) | (assets["maturity_years"] > MATURITY_MAX))].index
    if len(bad_mat) > 0:
        add_exc("ERROR","INVALID_MATURITY",
                f"{len(bad_mat)} rows have maturity_years outside 0–{MATURITY_MAX}.", bad_mat,
                fix="Fix maturity; ensure numeric years.")

# MV/BV divergence (warn)
ok = assets["book_value"].notna() & (assets["book_value"] > 0) & assets["market_value"].notna()
div = (assets.loc[ok, "market_value"] - assets.loc[ok, "book_value"]).abs() / assets.loc[ok, "book_value"]
bad_div = assets.loc[ok].index[div > MV_BV_OUTLIER_WARN]
if len(bad_div) > 0:
    add_exc("WARN","MV_BV_OUTLIER",
            f"{len(bad_div)} rows have |MV-BV|/BV > {int(MV_BV_OUTLIER_WARN*100)}%.", bad_div,
            fix="Review units/currency; confirm pricing vs amortized cost is expected.")

# 4.4 Mapping checks (rating + asset_type)
type_norm = assets["asset_type"].astype(str).str.lower().str.strip()
bad_type = assets[(type_norm!="") & (~type_norm.isin(ALLOWED_ASSET_TYPES))].index
if len(bad_type) > 0:
    add_exc("WARN","ASSET_TYPE_NOT_ALLOWED",
            f"{len(bad_type)} rows have asset_type not in {sorted(ALLOWED_ASSET_TYPES)}.", bad_type,
            fix="Normalize asset_type values (Bond/bond -> bond).")

rating_norm = assets["rating"].astype(str).str.upper().str.strip()
bad_rating = assets[(rating_norm!="") & (~rating_norm.isin(valid_ratings))].index
if len(bad_rating) > 0:
    add_exc("ERROR","RATING_NOT_MAPPED",
            f"{len(bad_rating)} rows have rating not mapped in credit_spreads_base.csv.", bad_rating,
            fix="Normalize ratings (AA-/AA+ -> AA) or expand spreads table.")

# ====== 5) Write exceptions_assets.csv ======
exceptions_df = pd.DataFrame(exceptions)
out_path = os.path.join(recon_path, "exceptions_assets.csv")
exceptions_df.to_csv(out_path, index=False)

display(exceptions_df)


✅ Phase 2 complete: wrote /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/reconciliation_pack/exceptions_assets.csv


,run_id,severity,issue_code,issue_detail,affected_rows,pct_of_file,sample_asset_ids,suggested_fix
0,2025Q4_MVP,ERROR,MISSING_ASSET_ID,3 rows have blank asset_id.,3,0.050000,"nan, nan, nan",Populate asset_id for every row (unique key).
1,2025Q4_MVP,ERROR,DUPLICATE_ASSET_ID,3 duplicated asset_id values found.,7,0.116667,"A0049, A0030, nan, nan, A0030",De-duplicate upstream or add lot/position suff...
2,2025Q4_MVP,ERROR,INVALID_MARKET_VALUE,3 rows have market_value missing or <= 0.,3,0.050000,"A0002, A0004, A0025",Fix pricing feed / ensure market_value parses ...
3,2025Q4_MVP,ERROR,INVALID_DURATION,3 rows have duration missing/negative/> 50.,3,0.050000,"A0036, A0051, nan","Fix duration feed; ensure numeric years, not t..."
4,2025Q4_MVP,WARN,MV_BV_OUTLIER,1 rows have |MV-BV|/BV > 50%.,1,0.016667,A0004,Review units/currency; confirm pricing vs amor...
5,2025Q4_MVP,WARN,ASSET_TYPE_NOT_ALLOWED,"1 rows have asset_type not in ['bond', 'cash'].",1,0.016667,A0025,Normalize asset_type values (Bond/bond -> bond).
6,2025Q4_MVP,ERROR,RATING_NOT_MAPPED,1 rows have rating not mapped in credit_spread...,1,0.016667,A0031,Normalize ratings (AA-/AA+ -> AA) or expand sp...


In [27]:
# Phase 3 — Assumption Governance (simple recruiter demo)
# Creates:
# 1) assumptions_library/<prior_qtr>/... (if missing, auto-generates a prior quarter set)
# 2) assumptions_library/<current_qtr>/... (saves your current base+shocked)
# 3) assumptions_diff_report.xlsx (tenor + rating diffs in bps)

!pip -q install pyyaml pandas numpy openpyxl

import os, yaml
import pandas as pd
import numpy as np
from datetime import datetime

# ---- 0) Set your Drive project path ----
PROJECT_DIR = "/content/drive/MyDrive/quarter_close_alm"

# ---- 1) Load config + locate current assumptions (from Phase 1) ----
config_path = os.path.join(PROJECT_DIR, "config", "run_config.yml")
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

run_id = cfg["run_id"]
out_root = cfg.get("paths", {}).get("outputs_root", "outputs")
run_path = os.path.join(PROJECT_DIR, out_root, run_id)
axis_path = os.path.join(run_path, "axis_like_inputs")

yc_base_path   = os.path.join(axis_path, "econ_yield_curve_base.csv")
yc_shock_path  = os.path.join(axis_path, "econ_yield_curve_shocked.csv")
cs_base_path   = os.path.join(axis_path, "credit_spreads_base.csv")
cs_shock_path  = os.path.join(axis_path, "credit_spreads_shocked.csv")

for p in [yc_base_path, yc_shock_path, cs_base_path, cs_shock_path]:
    assert os.path.exists(p), f"Missing current assumption file: {p}"

yc_cur_base  = pd.read_csv(yc_base_path)
yc_cur_shock = pd.read_csv(yc_shock_path)
cs_cur_base  = pd.read_csv(cs_base_path)
cs_cur_shock = pd.read_csv(cs_shock_path)

# ---- 2) Decide current quarter + prior quarter (simple, based on base_date) ----
def quarter_from_date(date_str: str) -> str:
    # date_str like "2025-12-31"
    y, m, _ = date_str.split("-")
    m = int(m)
    q = (m - 1)//3 + 1
    return f"{y}Q{q}"

def prior_quarter(qtr: str) -> str:
    y = int(qtr[:4]); q = int(qtr[-1])
    if q == 1:
        return f"{y-1}Q4"
    return f"{y}Q{q-1}"

current_qtr = quarter_from_date(cfg.get("base_date", "2025-12-31"))
prior_qtr = prior_quarter(current_qtr)

print("Current quarter:", current_qtr)
print("Prior quarter:", prior_qtr)

# ---- 3) Versioned assumptions library paths ----
lib_root = os.path.join(PROJECT_DIR, "assumptions_library")
cur_dir  = os.path.join(lib_root, current_qtr)
pri_dir  = os.path.join(lib_root, prior_qtr)
os.makedirs(cur_dir, exist_ok=True)
os.makedirs(pri_dir, exist_ok=True)

def save_set(folder, yc_base, yc_shock, cs_base, cs_shock):
    yc_base.to_csv(os.path.join(folder, "yield_curve_base.csv"), index=False)
    yc_shock.to_csv(os.path.join(folder, "yield_curve_shocked.csv"), index=False)
    cs_base.to_csv(os.path.join(folder, "credit_spreads_base.csv"), index=False)
    cs_shock.to_csv(os.path.join(folder, "credit_spreads_shocked.csv"), index=False)

# Save CURRENT quarter assumptions (governance evidence)
save_set(cur_dir, yc_cur_base, yc_cur_shock, cs_cur_base, cs_cur_shock)
print("Saved current assumptions to:", cur_dir)

# ---- 4) Ensure PRIOR quarter exists; if not, auto-generate a realistic prior set ----
pri_yc_base_path  = os.path.join(pri_dir, "yield_curve_base.csv")
pri_cs_base_path  = os.path.join(pri_dir, "credit_spreads_base.csv")
pri_yc_shock_path = os.path.join(pri_dir, "yield_curve_shocked.csv")
pri_cs_shock_path = os.path.join(pri_dir, "credit_spreads_shocked.csv")

def make_prior_from_current(yc_base, cs_base, seed=42):
    """
    Simple demo logic:
    - Prior quarter curve = current curve - small random shifts (5–20 bps)
    - Prior spreads = current spreads - small random shifts (3–15 bps)
    """
    rng = np.random.default_rng(seed)

    yc = yc_base.copy()
    curve_shift_bps = rng.integers(5, 21, size=len(yc))
    yc["zero_rate"] = yc["zero_rate"] - (curve_shift_bps / 10000.0)

    cs = cs_base.copy()
    spread_shift_bps = rng.integers(3, 16, size=len(cs))
    cs["spread_bps"] = cs["spread_bps"] - spread_shift_bps

    return yc, cs

if not (os.path.exists(pri_yc_base_path) and os.path.exists(pri_cs_base_path)):
    yc_pri_base, cs_pri_base = make_prior_from_current(yc_cur_base, cs_cur_base)
    # Build prior shocked using same shocks (for completeness)
    rate_shock = cfg["scenario"]["rate_shock_bps"]
    spread_shock = cfg["scenario"]["spread_shock_bps"]

    yc_pri_shock = yc_pri_base.copy()
    yc_pri_shock["zero_rate"] = yc_pri_shock["zero_rate"] + rate_shock/10000.0

    cs_pri_shock = cs_pri_base.copy()
    cs_pri_shock["spread_bps"] = cs_pri_shock["spread_bps"] + spread_shock

    save_set(pri_dir, yc_pri_base, yc_pri_shock, cs_pri_base, cs_pri_shock)
    print("Prior assumptions did not exist — generated and saved to:", pri_dir)

# Load prior assumptions (now guaranteed to exist)
yc_pri_base  = pd.read_csv(pri_yc_base_path)
cs_pri_base  = pd.read_csv(pri_cs_base_path)

# ---- 5) Build diffs (bps) ----
# Yield curve diff: join by tenor_years
yc_diff = (
    yc_pri_base.rename(columns={"zero_rate":"prior_zero_rate"})
    .merge(yc_cur_base.rename(columns={"zero_rate":"current_zero_rate"}), on="tenor_years", how="outer")
    .sort_values("tenor_years")
)
yc_diff["change_bps"] = (yc_diff["current_zero_rate"] - yc_diff["prior_zero_rate"]) * 10000
yc_diff["change_bps"] = yc_diff["change_bps"].round(1)

# Credit spread diff: join by rating
cs_diff = (
    cs_pri_base.rename(columns={"spread_bps":"prior_spread_bps"})
    .merge(cs_cur_base.rename(columns={"spread_bps":"current_spread_bps"}), on="rating", how="outer")
    .sort_values("rating")
)
cs_diff["change_bps"] = (cs_diff["current_spread_bps"] - cs_diff["prior_spread_bps"])
cs_diff["change_bps"] = cs_diff["change_bps"].round(1)

# Simple executive summary
yc_min = float(yc_diff["change_bps"].min())
yc_max = float(yc_diff["change_bps"].max())
cs_min = float(cs_diff["change_bps"].min())
cs_max = float(cs_diff["change_bps"].max())

top_curve_moves = yc_diff.reindex(yc_diff["change_bps"].abs().sort_values(ascending=False).index).head(3)
top_spread_moves = cs_diff.reindex(cs_diff["change_bps"].abs().sort_values(ascending=False).index).head(3)

exec_summary = pd.DataFrame([
    {"item":"Quarter compared", "value": f"{prior_qtr} → {current_qtr}"},
    {"item":"Curve change range (bps)", "value": f"{yc_min:.1f} to {yc_max:.1f}"},
    {"item":"Spread change range (bps)", "value": f"{cs_min:.1f} to {cs_max:.1f}"},
    {"item":"Generated at (UTC)", "value": datetime.utcnow().isoformat() + "Z"},
])

# ---- 6) Write report: assumptions_diff_report.xlsx ----
report_path = os.path.join(run_path, "assumptions_diff_report.xlsx")

with pd.ExcelWriter(report_path, engine="openpyxl") as xl:
    exec_summary.to_excel(xl, sheet_name="Executive_Summary", index=False)
    top_curve_moves.to_excel(xl, sheet_name="Top_Curve_Moves", index=False)
    top_spread_moves.to_excel(xl, sheet_name="Top_Spread_Moves", index=False)
    yc_diff.to_excel(xl, sheet_name="YieldCurve_Diff", index=False)
    cs_diff.to_excel(xl, sheet_name="CreditSpreads_Diff", index=False)

print("Report:", report_path)
print("Governed assumptions saved under:", lib_root)

# Optional: download the report in Colab
from google.colab import files
files.download(report_path)


Current quarter: 2025Q4
Prior quarter: 2025Q3
Saved current assumptions to: /content/drive/MyDrive/quarter_close_alm/assumptions_library/2025Q4
Report: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/assumptions_diff_report.xlsx
Governed assumptions saved under: /content/drive/MyDrive/quarter_close_alm/assumptions_library


/tmp/ipython-input-397517855.py:152: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  {"item":"Generated at (UTC)", "value": datetime.utcnow().isoformat() + "Z"},


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
# Phase 4 — Reinvestment Strategy (simple recruiter demo)
# Creates:
# - outputs/<run_id>/axis_like_inputs/reinvestment_plan.csv
# - outputs/<run_id>/axis_like_inputs/reinvestment_summary.csv
# - outputs/<run_id>/reinvestment_inputs.xlsx  (Reinvest_Summary + Reinvest_Plan)

!pip -q install pyyaml pandas numpy openpyxl

import os, yaml
import pandas as pd
import numpy as np
from datetime import datetime

# ---- 0) Set your Drive project path ----
PROJECT_DIR = "/content/drive/MyDrive/quarter_close_alm"

# ---- 1) Load config + locate inputs ----
config_path = os.path.join(PROJECT_DIR, "config", "run_config.yml")
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

run_id = cfg["run_id"]
out_root = cfg.get("paths", {}).get("outputs_root", "outputs")
run_path = os.path.join(PROJECT_DIR, out_root, run_id)

axis_path = os.path.join(run_path, "axis_like_inputs")
recon_path = os.path.join(run_path, "reconciliation_pack")
os.makedirs(axis_path, exist_ok=True)
os.makedirs(recon_path, exist_ok=True)

asset_file = os.path.join(axis_path, "asset_inventory.csv")
assert os.path.exists(asset_file), f"Missing: {asset_file}"

assets = pd.read_csv(asset_file)

# ---- 2) Policy rules (keep them simple + explainable) ----
# You can move these into cfg later; hard-coding is fine for recruiter demo.
policy = {
    "min_cash_pct": 0.03,         # liquidity reserve
    "max_below_ig_pct": 0.10,     # cap allocations to below IG (BB/B/NR)
    "duration_target": 5.0,       # portfolio duration target
    "duration_band": 1.0          # +/- band (so target is 4–6)
}

# Rating classification (simple)
IG = {"AAA","AA","A","BBB"}
BELOW_IG = {"BB","B","NR"}

# ---- 3) Determine cash available (demo logic) ----
# If you have actual cash holdings, use them; otherwise use a manual value.
assets["asset_type_norm"] = assets["asset_type"].astype(str).str.lower().str.strip()

cash_holdings = assets.loc[assets["asset_type_norm"] == "cash", "market_value"].fillna(0).sum()

# If there is no cash row, set a manual reinvestment amount (demo-friendly)
manual_cash_to_reinvest = 10_000_000.0  # $10m
cash_to_reinvest = float(cash_holdings) if cash_holdings > 0 else manual_cash_to_reinvest

liquidity_reserve_required = cash_to_reinvest * policy["min_cash_pct"]
investable_amount = cash_to_reinvest - liquidity_reserve_required

# ---- 4) Define allocation buckets (simple and explainable) ----
# Keep 4–5 buckets max.
# alloc_pct should sum to 1.0 of INVESTABLE AMOUNT (cash reserve handled separately).
buckets = pd.DataFrame([
    {"bucket_name":"Govt_AA_Intermediate", "sector":"GOVT", "rating":"AA",  "alloc_pct":0.30, "dur_min":4.0, "dur_max":6.0},
    {"bucket_name":"Corp_A_Core",          "sector":"CORP", "rating":"A",   "alloc_pct":0.45, "dur_min":4.0, "dur_max":7.0},
    {"bucket_name":"Corp_BBB_Long",        "sector":"CORP", "rating":"BBB", "alloc_pct":0.18, "dur_min":5.0, "dur_max":8.0},
    {"bucket_name":"HighYield_BB",         "sector":"CORP", "rating":"BB",  "alloc_pct":0.07, "dur_min":3.0, "dur_max":6.0},
])

# Validate allocations sum to 1.0
alloc_sum = float(buckets["alloc_pct"].sum())
if abs(alloc_sum - 1.0) > 1e-6:
    raise ValueError(f"Bucket alloc_pct must sum to 1.0. Current sum = {alloc_sum}")

# ---- 5) Apply policy checks + compute allocated amounts ----
buckets["alloc_amount"] = (buckets["alloc_pct"] * investable_amount).round(2)

# below-IG check
buckets["rating_norm"] = buckets["rating"].astype(str).str.upper().str.strip()
below_ig_alloc_pct = float(buckets[buckets["rating_norm"].isin(BELOW_IG)]["alloc_pct"].sum())
below_ig_pass = below_ig_alloc_pct <= policy["max_below_ig_pct"]

# duration target check (use weighted midpoint duration as a simple proxy)
buckets["dur_mid"] = (buckets["dur_min"] + buckets["dur_max"]) / 2.0
weighted_dur = float((buckets["alloc_pct"] * buckets["dur_mid"]).sum())
dur_low = policy["duration_target"] - policy["duration_band"]
dur_high = policy["duration_target"] + policy["duration_band"]
duration_pass = (weighted_dur >= dur_low) and (weighted_dur <= dur_high)

# Add policy flags per bucket (basic)
def bucket_flags(row):
    flags = []
    # flag below IG buckets if cap exceeded
    if row["rating_norm"] in BELOW_IG and not below_ig_pass:
        flags.append("BelowIGCapExceeded")
    # flag if outside target band (rare with our settings, but included)
    if not (row["dur_mid"] >= dur_low and row["dur_mid"] <= dur_high):
        flags.append("DurOutsideTargetBand")
    return "OK" if not flags else ";".join(flags)

buckets["policy_flags"] = buckets.apply(bucket_flags, axis=1)

# Final reinvestment plan (what you hand to modeling / AXIS-like inputs)
reinvestment_plan = buckets[[
    "bucket_name","sector","rating","dur_min","dur_max",
    "alloc_pct","alloc_amount","policy_flags"
]].copy()

# ---- 6) Build reinvestment summary (stakeholder-friendly) ----
reinvestment_summary = pd.DataFrame([
    {"item":"run_id", "value": run_id},
    {"item":"generated_at_utc", "value": datetime.utcnow().isoformat() + "Z"},
    {"item":"cash_to_reinvest", "value": round(cash_to_reinvest, 2)},
    {"item":"liquidity_reserve_pct", "value": policy["min_cash_pct"]},
    {"item":"liquidity_reserve_required", "value": round(liquidity_reserve_required, 2)},
    {"item":"investable_amount", "value": round(investable_amount, 2)},
    {"item":"max_below_ig_pct", "value": policy["max_below_ig_pct"]},
    {"item":"below_ig_alloc_pct", "value": round(below_ig_alloc_pct, 4)},
    {"item":"below_ig_policy_pass", "value": bool(below_ig_pass)},
    {"item":"duration_target", "value": policy["duration_target"]},
    {"item":"duration_band", "value": policy["duration_band"]},
    {"item":"weighted_duration_proxy", "value": round(weighted_dur, 3)},
    {"item":"duration_policy_pass", "value": bool(duration_pass)},
])

# ---- 7) Write outputs (CSV + Excel) ----
plan_csv = os.path.join(axis_path, "reinvestment_plan.csv")
summary_csv = os.path.join(axis_path, "reinvestment_summary.csv")

reinvestment_plan.to_csv(plan_csv, index=False)
reinvestment_summary.to_csv(summary_csv, index=False)

excel_path = os.path.join(run_path, "reinvestment_inputs.xlsx")
with pd.ExcelWriter(excel_path, engine="openpyxl") as xl:
    reinvestment_summary.to_excel(xl, sheet_name="Reinvest_Summary", index=False)
    reinvestment_plan.to_excel(xl, sheet_name="Reinvest_Plan", index=False)

print("Plan CSV:", plan_csv)
print("Summary CSV:", summary_csv)
print("Excel:", excel_path)

# Optional: download Excel
from google.colab import files
files.download(excel_path)

reinvestment_plan


Plan CSV: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/axis_like_inputs/reinvestment_plan.csv
Summary CSV: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/axis_like_inputs/reinvestment_summary.csv
Excel: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/reinvestment_inputs.xlsx


/tmp/ipython-input-4163564859.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  {"item":"generated_at_utc", "value": datetime.utcnow().isoformat() + "Z"},


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,bucket_name,sector,rating,dur_min,dur_max,alloc_pct,alloc_amount,policy_flags
0,Govt_AA_Intermediate,GOVT,AA,4.0,6.0,0.30,2910000.0,OK
1,Corp_A_Core,CORP,A,4.0,7.0,0.45,4365000.0,OK
2,Corp_BBB_Long,CORP,BBB,5.0,8.0,0.18,1746000.0,DurOutsideTargetBand
3,HighYield_BB,CORP,BB,3.0,6.0,0.07,679000.0,OK


In [32]:
# Phase 5 — Attribution + Reconciliation (recruiter demo, tied to your Phase 1 outputs)
# Fixes the pandas Series -> float TypeError and is messy-data safe.
#
# Reads (from previous steps):
# - outputs/<run_id>/reconciliation_pack/kpis.csv
# - outputs/<run_id>/axis_like_inputs/asset_inventory.csv
# - outputs/<run_id>/axis_like_inputs/econ_yield_curve_base.csv
# - outputs/<run_id>/axis_like_inputs/credit_spreads_base.csv  (not strictly required, but kept for consistency)
#
# Writes:
# - outputs/<run_id>/reconciliation_pack/attribution_summary.csv
# - Adds/updates "Attribution" sheet in outputs/<run_id>/kpi_dashboard.xlsx

!pip -q install pyyaml pandas numpy openpyxl

import os, yaml
import pandas as pd
import numpy as np
from datetime import datetime

# ---- 0) Set your Drive project path ----
PROJECT_DIR = "/content/drive/MyDrive/quarter_close_alm"

# ---- 1) Load config + locate run paths ----
config_path = os.path.join(PROJECT_DIR, "config", "run_config.yml")
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

run_id = cfg["run_id"]
out_root = cfg.get("paths", {}).get("outputs_root", "outputs")
run_path = os.path.join(PROJECT_DIR, out_root, run_id)

axis_path  = os.path.join(run_path, "axis_like_inputs")
recon_path = os.path.join(run_path, "reconciliation_pack")
os.makedirs(recon_path, exist_ok=True)

asset_file   = os.path.join(axis_path, "asset_inventory.csv")
yc_base_file  = os.path.join(axis_path, "econ_yield_curve_base.csv")
cs_base_file  = os.path.join(axis_path, "credit_spreads_base.csv")
kpis_file     = os.path.join(recon_path, "kpis.csv")

for p in [asset_file, yc_base_file, cs_base_file, kpis_file]:
    assert os.path.exists(p), f"Missing required file: {p}"

assets = pd.read_csv(asset_file)
yc_base = pd.read_csv(yc_base_file)
cs_base = pd.read_csv(cs_base_file)
kpis = pd.read_csv(kpis_file)

# Scenario shocks from your config (same ones used in Phase 1)
rate_shock_bps = float(cfg["scenario"]["rate_shock_bps"])
spread_shock_bps = float(cfg["scenario"]["spread_shock_bps"])

# ---- 2) Pull base/scenario totals from your Phase 1 KPIs ----
def get_metric(df, metric_name: str):
    df2 = df.copy()
    df2["metric"] = df2["metric"].astype(str)
    row = df2[df2["metric"] == metric_name]
    if row.empty:
        raise ValueError(f"Metric not found in kpis.csv: {metric_name}")
    return float(row["base"].iloc[0]), float(row["scenario"].iloc[0])

base_assets_mv, scen_assets_mv = get_metric(kpis, "Assets_MV")
base_liab_pv, scen_liab_pv     = get_metric(kpis, "Liability_PV")
base_surplus, scen_surplus     = get_metric(kpis, "Surplus")

delta_surplus_total = scen_surplus - base_surplus

# ---- 3) Attribution logic (simple but reconciles exactly) ----
# Assets:
#   rate impact   ~ -Dur * Δr * MV
#   spread impact ~ -Dur * Δs * MV
#
# Liabilities:
#   compute PV using base curve vs (base curve + rate shock) => rate PV change
#   surplus effect = -ΔLiabPV
#
# Carry proxy:
#   quarterly carry based on 1Y rate * (base surplus)
#
# Residual:
#   difference so that components sum exactly to total surplus change

# --- Messy-data safe numeric coercion (KEY FIX) ---
assets_mv  = pd.to_numeric(assets.get("market_value"), errors="coerce").fillna(0.0)
assets_dur = pd.to_numeric(assets.get("mod_duration"), errors="coerce").fillna(0.0)

dy_rate = rate_shock_bps / 10000.0
dy_spread = spread_shock_bps / 10000.0

# ✅ Corrected: SUM FIRST, then float (prevents TypeError)
asset_rate_impact   = float(((-assets_dur * dy_rate)   * assets_mv).sum())
asset_spread_impact = float(((-assets_dur * dy_spread) * assets_mv).sum())

# Liability PV function (same style as Phase 1 MVP)
def liability_pv_from_curve(yc_df: pd.DataFrame) -> float:
    # 10 annual payments of 10m (proxy cashflows)
    t = np.arange(1, 11)
    cf = np.array([10_000_000] * 10, dtype=float)

    ten = pd.to_numeric(yc_df["tenor_years"], errors="coerce").values
    r = pd.to_numeric(yc_df["zero_rate"], errors="coerce").values
    # Interpolate rate to each year
    r_t = np.interp(t, ten, r)
    df = 1 / ((1 + r_t) ** t)
    return float((cf * df).sum())

yc_shocked = yc_base.copy()
yc_shocked["zero_rate"] = pd.to_numeric(yc_shocked["zero_rate"], errors="coerce") + dy_rate

liab_pv_recalc_base = liability_pv_from_curve(yc_base)
liab_pv_recalc_shock = liability_pv_from_curve(yc_shocked)

liab_rate_pv_change = liab_pv_recalc_shock - liab_pv_recalc_base
surplus_liab_rate_impact = -liab_rate_pv_change  # Surplus = Assets - Liab

# Carry proxy (quarterly)
# Use ~1Y rate from base curve as a simple carry estimate
yc_tmp = yc_base.copy()
yc_tmp["tenor_years"] = pd.to_numeric(yc_tmp["tenor_years"], errors="coerce")
yc_tmp["zero_rate"] = pd.to_numeric(yc_tmp["zero_rate"], errors="coerce")
one_year_idx = (yc_tmp["tenor_years"] - 1).abs().idxmin()
one_year_rate = float(yc_tmp.loc[one_year_idx, "zero_rate"])

time_fraction = 0.25
carry_proxy = (base_surplus) * one_year_rate * time_fraction

components = [
    ("Asset rate impact", asset_rate_impact),
    ("Asset spread impact", asset_spread_impact),
    ("Liability rate impact (surplus effect)", surplus_liab_rate_impact),
    ("Cashflow/interest carry (proxy)", carry_proxy),
]

explained = sum(v for _, v in components)
residual = delta_surplus_total - explained

# ---- 4) Output table (stakeholder-friendly and reconciles) ----
attrib = pd.DataFrame(
    [{"component": name, "amount": float(val)} for name, val in components] +
    [{"component": "Residual (unexplained / rounding)", "amount": float(residual)},
     {"component": "Total surplus change (Scenario - Base)", "amount": float(delta_surplus_total)}]
)
attrib["amount"] = attrib["amount"].round(2)

attrib_context = pd.DataFrame([{
    "run_id": run_id,
    "scenario_name": cfg["scenario"]["name"],
    "rate_shock_bps": rate_shock_bps,
    "spread_shock_bps": spread_shock_bps,
    "base_surplus": round(base_surplus, 2),
    "scenario_surplus": round(scen_surplus, 2),
    "total_surplus_change": round(delta_surplus_total, 2),
    "generated_at_utc": datetime.utcnow().isoformat() + "Z"
}])

attrib_csv = os.path.join(recon_path, "attribution_summary.csv")
attrib.to_csv(attrib_csv, index=False)

print("✅ Phase 5: attribution_summary.csv written:", attrib_csv)
display(attrib_context)
display(attrib)

# ---- 5) Add / update Attribution sheet in KPI dashboard ----
dash_path = os.path.join(run_path, "kpi_dashboard.xlsx")

# If dashboard doesn't exist (unlikely since you did Phase 1.6), create a minimal one
if not os.path.exists(dash_path):
    with pd.ExcelWriter(dash_path, engine="openpyxl") as xl:
        kpis.to_excel(xl, sheet_name="Summary_KPIs", index=False)

# Replace or create Attribution sheet
with pd.ExcelWriter(dash_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as xl:
    attrib_context.to_excel(xl, sheet_name="Attribution", index=False, startrow=0)
    attrib.to_excel(xl, sheet_name="Attribution", index=False, startrow=4)


# Optional: download updated dashboard
from google.colab import files
files.download(dash_path)


✅ Phase 5: attribution_summary.csv written: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/reconciliation_pack/attribution_summary.csv


/tmp/ipython-input-3159998509.py:154: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "generated_at_utc": datetime.utcnow().isoformat() + "Z"


,run_id,scenario_name,rate_shock_bps,spread_shock_bps,base_surplus,scenario_surplus,total_surplus_change,generated_at_utc
0,2025Q4_MVP,RatesUp100_SpreadsUp50,100.0,50.0,93825578.71,87169192.78,-6656385.93,2025-12-22T00:14:59.214767Z


,component,amount
0,Asset rate impact,-6965189.11
1,Asset spread impact,-3482594.55
2,Liability rate impact (surplus effect),3791397.73
3,Cashflow/interest carry (proxy),1125906.94
4,Residual (unexplained / rounding),-1125906.94
5,Total surplus change (Scenario - Base),-6656385.93


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
# Phase 6 — Close Pack (Recruiter-ready bundle)
# Creates:
# - outputs/<run_id>/quarter_close_pack_<YYYYQ#>/ (folder)
#   - executive_summary.md
#   - controls_evidence/controls_evidence_checklist.csv
#   - controls_evidence/run_manifest.csv
#   - run_log.txt
#   - axis_like_inputs/ (copied)
#   - reconciliation_pack/ (copied)
#   - kpi_dashboard.xlsx (copied)
#   - assumptions_diff_report.xlsx (copied if exists)
#   - reinvestment_inputs.xlsx (copied if exists)
# - outputs/<run_id>/quarter_close_pack_<YYYYQ#>.zip  (downloadable)

!pip -q install pyyaml pandas numpy openpyxl

import os, yaml, shutil, zipfile, hashlib
import pandas as pd
from datetime import datetime

# ---- 0) Set your Drive project path ----
PROJECT_DIR = "/content/drive/MyDrive/quarter_close_alm"

# ---- 1) Load config + paths ----
config_path = os.path.join(PROJECT_DIR, "config", "run_config.yml")
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

run_id = cfg["run_id"]
out_root = cfg.get("paths", {}).get("outputs_root", "outputs")
run_path = os.path.join(PROJECT_DIR, out_root, run_id)

axis_path  = os.path.join(run_path, "axis_like_inputs")
recon_path = os.path.join(run_path, "reconciliation_pack")

kpi_dash_path = os.path.join(run_path, "kpi_dashboard.xlsx")
assump_diff_path = os.path.join(run_path, "assumptions_diff_report.xlsx")
reinv_xlsx_path = os.path.join(run_path, "reinvestment_inputs.xlsx")

kpis_csv = os.path.join(recon_path, "kpis.csv")
exceptions_csv = os.path.join(recon_path, "exceptions_assets.csv")
attrib_csv = os.path.join(recon_path, "attribution_summary.csv")

# ---- 2) Determine quarter label (e.g., 2025Q4) ----
def quarter_from_date(date_str: str) -> str:
    y, m, _ = date_str.split("-")
    m = int(m)
    q = (m - 1)//3 + 1
    return f"{y}Q{q}"

quarter_label = quarter_from_date(cfg.get("base_date", "2025-12-31"))
pack_folder_name = f"quarter_close_pack_{quarter_label}"
pack_dir = os.path.join(run_path, pack_folder_name)
controls_dir = os.path.join(pack_dir, "controls_evidence")

os.makedirs(pack_dir, exist_ok=True)
os.makedirs(controls_dir, exist_ok=True)

print("RUN_ID:", run_id)
print("Quarter label:", quarter_label)
print("Pack folder:", pack_dir)

# ---- 3) Helpers ----
def sha256_file(path, chunk_size=2**20):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def safe_copy(src, dst):
    if os.path.exists(src):
        shutil.copy2(src, dst)
        return True
    return False

def safe_copytree(src_dir, dst_dir):
    if os.path.exists(src_dir):
        if os.path.exists(dst_dir):
            shutil.rmtree(dst_dir)
        shutil.copytree(src_dir, dst_dir)
        return True
    return False

# ---- 4) Copy main deliverables into the pack ----
copied = []
copied.append(("axis_like_inputs", safe_copytree(axis_path, os.path.join(pack_dir, "axis_like_inputs"))))
copied.append(("reconciliation_pack", safe_copytree(recon_path, os.path.join(pack_dir, "reconciliation_pack"))))

copied.append(("kpi_dashboard.xlsx", safe_copy(kpi_dash_path, os.path.join(pack_dir, "kpi_dashboard.xlsx"))))
copied.append(("assumptions_diff_report.xlsx", safe_copy(assump_diff_path, os.path.join(pack_dir, "assumptions_diff_report.xlsx"))))
copied.append(("reinvestment_inputs.xlsx", safe_copy(reinv_xlsx_path, os.path.join(pack_dir, "reinvestment_inputs.xlsx"))))

# ---- 5) Build run manifest (controls evidence) ----
manifest_rows = []

def add_manifest_item(name, path):
    if os.path.exists(path) and os.path.isfile(path):
        manifest_rows.append({
            "run_id": run_id,
            "file_name": name,
            "relative_path": os.path.relpath(path, run_path),
            "sha256": sha256_file(path),
            "bytes": os.path.getsize(path),
            "generated_at_utc": datetime.utcnow().isoformat() + "Z"
        })

# include key outputs
add_manifest_item("kpi_dashboard.xlsx", kpi_dash_path)
add_manifest_item("assumptions_diff_report.xlsx", assump_diff_path)
add_manifest_item("reinvestment_inputs.xlsx", reinv_xlsx_path)
add_manifest_item("kpis.csv", kpis_csv)
add_manifest_item("exceptions_assets.csv", exceptions_csv)
add_manifest_item("attribution_summary.csv", attrib_csv)

manifest_df = pd.DataFrame(manifest_rows)
manifest_path = os.path.join(controls_dir, "run_manifest.csv")
manifest_df.to_csv(manifest_path, index=False)

# ---- 6) Controls evidence checklist (simple tick-box) ----
# Pull quick counts if files exist
errors_count = None
warns_count = None
if os.path.exists(exceptions_csv):
    exc = pd.read_csv(exceptions_csv)
    if "severity" in exc.columns:
        errors_count = int((exc["severity"] == "ERROR").sum())
        warns_count = int((exc["severity"] == "WARN").sum())

checklist = [
    ("Inputs folder exists (axis_like_inputs)", os.path.exists(axis_path)),
    ("Reconciliation folder exists (reconciliation_pack)", os.path.exists(recon_path)),
    ("KPI dashboard created", os.path.exists(kpi_dash_path)),
    ("KPIs file created (kpis.csv)", os.path.exists(kpis_csv)),
    ("Exceptions file created (exceptions_assets.csv)", os.path.exists(exceptions_csv)),
    ("Attribution created (attribution_summary.csv)", os.path.exists(attrib_csv)),
    ("Assumption diff report created (Phase 3)", os.path.exists(assump_diff_path)),
    ("Reinvestment inputs created (Phase 4)", os.path.exists(reinv_xlsx_path)),
]

# Attribution reconciles? (component sum should equal total change; we designed it that way)
attrib_reconciles = None
if os.path.exists(attrib_csv):
    a = pd.read_csv(attrib_csv)
    # if the total row exists, check it
    if "component" in a.columns and "amount" in a.columns:
        total_row = a[a["component"].astype(str).str.contains("Total surplus change", na=False)]
        resid_row = a[a["component"].astype(str).str.contains("Residual", na=False)]
        attrib_reconciles = (len(total_row) == 1) and (len(resid_row) == 1)

checklist.append(("Attribution reconciles to total surplus change", attrib_reconciles if attrib_reconciles is not None else False))

checklist_df = pd.DataFrame([{
    "run_id": run_id,
    "check_name": name,
    "status": "PASS" if ok else "FAIL",
    "details": (f"errors={errors_count}, warns={warns_count}" if "Exceptions" in name and errors_count is not None else "")
} for name, ok in checklist])

checklist_path = os.path.join(controls_dir, "controls_evidence_checklist.csv")
checklist_df.to_csv(checklist_path, index=False)

# ---- 7) Executive summary (1 page markdown) ----
# Pull KPI numbers if available
assets_mv_base = assets_mv_scen = liab_base = liab_scen = surplus_base = surplus_scen = None
scenario_name = cfg["scenario"]["name"]

if os.path.exists(kpis_csv):
    k = pd.read_csv(kpis_csv)
    def metric(m):
        r = k[k["metric"] == m]
        if r.empty:
            return None, None
        return float(r["base"].iloc[0]), float(r["scenario"].iloc[0])
    assets_mv_base, assets_mv_scen = metric("Assets_MV")
    liab_base, liab_scen = metric("Liability_PV")
    surplus_base, surplus_scen = metric("Surplus")

# Pull top assumption moves if the diff report exists (we’ll summarize lightly)
assump_note = "Assumption diff report not found (Phase 3 not run)."
if os.path.exists(assump_diff_path):
    assump_note = "See assumptions_diff_report.xlsx for curve/spread bps movements (YieldCurve_Diff and CreditSpreads_Diff)."

# Pull top attribution driver if exists
attrib_note = "Attribution not found (Phase 5 not run)."
if os.path.exists(attrib_csv):
    a = pd.read_csv(attrib_csv)
    # ignore residual + total rows
    core = a[~a["component"].astype(str).str.contains("Residual|Total", na=False)].copy()
    if not core.empty:
        top = core.reindex(core["amount"].abs().sort_values(ascending=False).index).head(1)
        attrib_note = f"Top driver: {top['component'].iloc[0]} = {float(top['amount'].iloc[0]):,.2f}"

exec_md = f"""# Quarterly Close Pack — {quarter_label}

## Run Metadata
- Run ID: **{run_id}**
- Scenario: **{scenario_name}**
- Generated (UTC): **{datetime.utcnow().isoformat()}Z**

## Key KPIs (Base vs Scenario)
- Assets MV: {assets_mv_base:,.2f} → {assets_mv_scen:,.2f}  (Δ {((assets_mv_scen-assets_mv_base) if assets_mv_base is not None else 0):,.2f})
- Liability PV: {liab_base:,.2f} → {liab_scen:,.2f}  (Δ {((liab_scen-liab_base) if liab_base is not None else 0):,.2f})
- Surplus: {surplus_base:,.2f} → {surplus_scen:,.2f}  (Δ {((surplus_scen-surplus_base) if surplus_base is not None else 0):,.2f})

## Assumptions (Governance)
- {assump_note}

## Reinvestment Strategy (Policy Overview)
- Liquidity reserve: keep **≥ 3%** cash (demo policy)
- Max below investment grade: **≤ 10%** allocation (demo policy)
- Duration target: **~5 years** within a band (demo policy)
- See reinvestment_inputs.xlsx for plan + summary.

## Attribution (Why surplus changed)
- {attrib_note}
- See Attribution sheet in kpi_dashboard.xlsx or attribution_summary.csv.

## Controls Evidence
- controls_evidence/controls_evidence_checklist.csv
- controls_evidence/run_manifest.csv
"""

exec_path = os.path.join(pack_dir, "executive_summary.md")
with open(exec_path, "w") as f:
    f.write(exec_md)

# ---- 8) Run log ----
run_log_lines = [
    f"RUN LOG — {quarter_label}",
    f"Run ID: {run_id}",
    f"Scenario: {scenario_name}",
    f"UTC time: {datetime.utcnow().isoformat()}Z",
    "",
    "Files copied into pack:",
]
for name, ok in copied:
    run_log_lines.append(f" - {name}: {'OK' if ok else 'MISSING'}")

if errors_count is not None:
    run_log_lines.append("")
    run_log_lines.append(f"Exceptions summary: ERROR checks triggered = {errors_count}, WARN checks triggered = {warns_count}")

run_log_path = os.path.join(pack_dir, "run_log.txt")
with open(run_log_path, "w") as f:
    f.write("\n".join(run_log_lines))

print("✅ Wrote executive summary:", exec_path)
print("✅ Wrote controls checklist:", checklist_path)
print("✅ Wrote run manifest:", manifest_path)
print("✅ Wrote run log:", run_log_path)

# ---- 9) Zip the whole close pack folder ----
zip_path = os.path.join(run_path, f"{pack_folder_name}.zip")

# If zip exists, overwrite
if os.path.exists(zip_path):
    os.remove(zip_path)

with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for root, dirs, files in os.walk(pack_dir):
        for file in files:
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, run_path)  # keep relative to run folder
            z.write(full_path, rel_path)

print("✅ Close pack zip created:", zip_path)

# Optional: download the zip
from google.colab import files
files.download(zip_path)


RUN_ID: 2025Q4_MVP
Quarter label: 2025Q4
Pack folder: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/quarter_close_pack_2025Q4


/tmp/ipython-input-1817212561.py:108: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "generated_at_utc": datetime.utcnow().isoformat() + "Z"
/tmp/ipython-input-1817212561.py:202: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  - Generated (UTC): **{datetime.utcnow().isoformat()}Z**
/tmp/ipython-input-1817212561.py:236: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f"UTC time: {datetime.utcnow().isoformat()}Z",


✅ Wrote executive summary: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/quarter_close_pack_2025Q4/executive_summary.md
✅ Wrote controls checklist: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/quarter_close_pack_2025Q4/controls_evidence/controls_evidence_checklist.csv
✅ Wrote run manifest: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/quarter_close_pack_2025Q4/controls_evidence/run_manifest.csv
✅ Wrote run log: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/quarter_close_pack_2025Q4/run_log.txt
✅ Close pack zip created: /content/drive/MyDrive/quarter_close_alm/outputs/2025Q4_MVP/quarter_close_pack_2025Q4.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>